In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
import os
import sys
from utils.utils import (
    print_with_colors,
)

# Legacy:

In [ ]:
if not os.path.exists("./raw_data"):
    os.makedirs("./raw_data")

if not os.path.exists("./raw_data/arbovirus_clinical_data"):
    # Download of .zip file
    url = "https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/2d3kr8zynf-4.zip"
    output = f"./raw_data/dataset.zip"
    subprocess.run(["wget", "--quiet", "--no-check-certificate", url, "-O", output])

    # Extraction of .zip file
    subprocess.run(["unzip", output])
    subprocess.run(["mv", "./2d3kr8zynf-4", "./raw_data/arbovirus_clinical_data"])
    subprocess.run(["rm", output])

In [2]:
missing_values = [
    '', ' ', 'NA', 'N/A', 'NULL',
    'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT',
    'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI',
    'DT_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
    'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS',
    'DT_INVEST', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA',
    'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE',
    'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO',
    'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
    'ACIDO_PEPT', 'AUTO_IMUNE', 'RESUL_SORO', 'RESUL_NS1', 'RESUL_VI_N',
    'RESUL_PCR_', 'HISTOPA_N', 'IMUNOH_N', 'HOSPITALIZ', 'TPAUTOCTO',
    'COUFINF', 'COPAISINF', 'COMUNINF', 'CLASSI_FIN', 'EVOLUCAO', 'DT_ENCERRA'
]

In [ ]:
# Low memory safe reading of the CSV file
splitted_df = pd.read_csv(
    './raw_data/arbovirus_clinical_data/data set.csv',
    sep=',',
    header=0,
    na_values=missing_values,
    chunksize=100_000, 
)

# Concatenate all chunks into a single DataFrame
concat_df = pd.concat(splitted_df, ignore_index=True)

concat_df = concat_df.dropna()

In [ ]:
for col in concat_df.columns:
    if concat_df[col].unique().size < 100:
        print(f"Column '{col}' has {concat_df[col].unique().size} unique values.")
        print(concat_df[col].unique(), end="\n\n")
    else:
        print_with_colors(f"Column '{col}' has {concat_df[col].unique().size} unique values. Skipping display...", "yellow", end="\n\n")

In [3]:
dtypes = {
    'ID_AGRAVO': 'category',
    'SEM_NOT': 'Int32',
    'NU_ANO': 'Int16',
    'SG_UF_NOT': 'category',
    'ID_MUNICIP': 'category',
    'ID_REGIONA': 'category',
    'ID_UNIDADE': 'category',
    'SEM_PRI': 'Int32',
    'NU_IDADE_N': 'Int8',
    'CS_SEXO': 'category',
    'CS_GESTANT': 'category',
    'CS_RACA': 'category',
    'CS_ESCOL_N': 'category',
    'SG_UF': 'category',
    'ID_MN_RESI': 'category',
    'ID_RG_RESI': 'category',
    'ID_PAIS': 'category',
    'FEBRE': 'category',
    'MIALGIA': 'category',
    'CEFALEIA': 'category',
    'EXANTEMA': 'category',
    'VOMITO': 'category',
    'NAUSEA': 'category',
    'DOR_COSTAS': 'category',
    'CONJUNTVIT': 'category',
    'ARTRITE': 'category',
    'ARTRALGIA': 'category',
    'PETEQUIA_N': 'category',
    'LEUCOPENIA': 'category',
    'LACO': 'category',
    'DOR_RETRO': 'category',
    'DIABETES': 'category',
    'HEMATOLOG': 'category',
    'HEPATOPAT': 'category',
    'RENAL': 'category',
    'HIPERTENSA': 'category',
    'ACIDO_PEPT': 'category',
    'AUTO_IMUNE': 'category',
    'RESUL_SORO': 'category',
    'RESUL_NS1': 'category',
    'RESUL_VI_N': 'category',
    'RESUL_PCR_': 'category',
    'HISTOPA_N': 'category',
    'IMUNOH_N': 'category',
    'HOSPITALIZ': 'category',
    'TPAUTOCTO': 'category',
    'COUFINF': 'category',
    'COPAISINF': 'category',
    'COMUNINF': 'category',
    'CLASSI_FIN': 'object',
    'EVOLUCAO': 'category',
    'DT_ENCERRA': 'datetime64[ns]',
    'DT_NOTIFIC': 'datetime64[ns]',
    'DT_SIN_PRI': 'datetime64[ns]',
    'DT_NASC': 'datetime64[ns]',
    'DT_INVEST': 'datetime64[ns]',
}

In [ ]:
colunas_data = ['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_NASC', 'DT_INVEST', 'DT_ENCERRA']
for col in colunas_data:
    if col in concat_df.columns:
        concat_df[col] = pd.to_datetime(concat_df[col], errors='coerce')

In [ ]:
problematical_columns = []
for col in concat_df.columns:
    try:
        if col in dtypes.keys():
            concat_df[col] = concat_df[col].astype(dtypes[col], errors='raise')
            print(f"Column '{col}' converted to {dtypes[col]}.", end="\n\n")
        else:
            print_with_colors(f"Column '{col}' not in dtypes dictionary. Skipping conversion...", "yellow", end="\n\n")
            problematical_columns.append(col)
    except TypeError as e:
        print(f"Error converting column '{col}': {e}", "red", end="\n\n")
        problematical_columns.append(col)
    except Exception as e:
        print_with_colors(f"Unexpected error converting column '{col}': {e}", "red", end="\n\n")
        problematical_columns.append(col)

# New:

In [3]:
import warnings

missing_values = [
    '', ' ', 'NA', 'N/A', 'NULL',
    'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT',
    'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI',
    'DT_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
    'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS',
    'DT_INVEST', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA',
    'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE',
    'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO',
    'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
    'ACIDO_PEPT', 'AUTO_IMUNE', 'RESUL_SORO', 'RESUL_NS1', 'RESUL_VI_N',
    'RESUL_PCR_', 'HISTOPA_N', 'IMUNOH_N', 'HOSPITALIZ', 'TPAUTOCTO',
    'COUFINF', 'COPAISINF', 'COMUNINF', 'CLASSI_FIN', 'EVOLUCAO', 'DT_ENCERRA'
]

warnings.filterwarnings("ignore")
# Low memory safe reading of the CSV file
splitted_df = pd.read_csv(
    './raw_data/arbovirus_clinical_data/dengue.csv',
    sep=',',
    header=0,
    na_values=missing_values,
    chunksize=100_000,
)

# Concatenate all chunks into a single DataFrame
dengue_df = pd.concat(splitted_df, ignore_index=True)
warnings.filterwarnings("default")

In [4]:
attributes = pd.read_csv("raw_data/arbovirus_clinical_data/attributes.csv", sep=",", header=0, low_memory=False)
attributes = attributes.ffill()
attributes = attributes.groupby(["Attribute", "Description"])["Value"].apply('; '.join).reset_index(name="Values")

## Pre Processing
---
### Null Data Removal:
Features with frequency > 60% of null values are dropped.

In [6]:
dengue_df = dengue_df.loc[:, dengue_df.isnull().mean() < .60]
dengue_df = dengue_df.drop(columns=["CS_FLXRET", "TP_SISTEMA", "CRITERIO", "TP_NOT", "Unnamed: 0"])

In [7]:
for col in dengue_df.columns.to_list():
    if str(col) in attributes["Attribute"].to_list():
        print(f"Column '{col}' has {dengue_df[col].unique().size} unique values.")
        if dengue_df[col].unique().size < 50:
            print(dengue_df[col].unique(), end="\n\n")
        else:
            print("To many unique values, skipping...", end="\n\n")
    else:
        print_with_colors(f"Column '{col}' not in attributes. Skipping display...", "yellow", end="\n\n")

Column 'ID_AGRAVO' has 1 unique values.
['A90']

Column 'DT_NOTIFIC' has 2926 unique values.
To many unique values, skipping...

Column 'SEM_NOT' has 451 unique values.
To many unique values, skipping...

Column 'NU_ANO' has 10 unique values.
[2018 2017 2013 2012 2014 2015 2019 2020 2021 2016]

Column 'SG_UF_NOT' has 27 unique values.
[25 26 24 31 32 53 41 35 23 51 12 43 11 52 50 15 13 42 28 27 29 17 16 21
 33 14 22]

Column 'ID_MUNICIP' has 5348 unique values.
To many unique values, skipping...

Column 'ID_REGIONA' has 343 unique values.
To many unique values, skipping...

Column 'ID_UNIDADE' has 53452 unique values.
To many unique values, skipping...

Column 'DT_SIN_PRI' has 17161 unique values.
To many unique values, skipping...

Column 'SEM_PRI' has 4437 unique values.
To many unique values, skipping...

Column 'DT_NASC' has 38705 unique values.
To many unique values, skipping...

Column 'NU_IDADE_N' has 514 unique values.
To many unique values, skipping...

Column 'CS_SEXO' has 4 

### Standardization of column values:

In [18]:
dengue_df['CLASSI_FIN'] = dengue_df['CLASSI_FIN'].astype('object')

dengue_df.loc[dengue_df['CLASSI_FIN']==1, 'CLASSI_FIN'] = 'Dengue'
dengue_df.loc[dengue_df['CLASSI_FIN']==10, 'CLASSI_FIN'] = 'Dengue'

dengue_df.loc[dengue_df['CLASSI_FIN']==3, 'CLASSI_FIN'] = 'Dengue Grave'
dengue_df.loc[dengue_df['CLASSI_FIN']==4, 'CLASSI_FIN'] = 'Dengue Grave'
dengue_df.loc[dengue_df['CLASSI_FIN']==12, 'CLASSI_FIN'] = 'Dengue Grave'

dengue_df.loc[dengue_df['CLASSI_FIN']==2, 'CLASSI_FIN'] = 'Dengue com sinais de alarme'
dengue_df.loc[dengue_df['CLASSI_FIN']==11, 'CLASSI_FIN'] = 'Dengue com sinais de alarme'

# Discarded/Inconclusive
dengue_df.loc[dengue_df['CLASSI_FIN']==5, 'CLASSI_FIN'] = 'Discarded/Inconclusive'
dengue_df.loc[dengue_df['CLASSI_FIN']==6, 'CLASSI_FIN'] = 'Discarded/Inconclusive'
dengue_df.loc[dengue_df['CLASSI_FIN']==8, 'CLASSI_FIN'] = 'Discarded/Inconclusive'

dengue_df['CLASSI_FIN'] = dengue_df['CLASSI_FIN'].fillna('Discarded/Inconclusive')
dengue_df['CLASSI_FIN'] = dengue_df['CLASSI_FIN'].astype('category')

### Null data padding with default values:
The resulting attributes that still had null data were entered with the default values referring to the data dictionary.

In [ ]:
exam_cols = [
    "RESUL_SORO",
    "RESUL_NS1",
    "RESUL_VI_N",
    "RESUL_PCR_",
    "HISTOPA_N",
    "IMUNOH_N"
]
for col in exam_cols:
    if dengue_df[col].isnull().sum() > 0:
        dengue_df.loc[dengue_df[col].isnull(), col] = 4

dengue_df['CS_SEXO'] = dengue_df['CS_SEXO'].fillna('I')

# In the other attributes, the value of "not informed" is 9.
columns_to_be_filled = [
    col
    for col in dengue_df.columns
    if col not in exam_cols
    and 'DT_' not in str(col) # for datetime columns it doesn't make sense
    and not 'CS_SEXO'.__eq__(str(col)) # CS_SEXO has the special value 'I' for NaNs
]
for col in columns_to_be_filled:
    if dengue_df[col].isnull().sum() > 0:
        dengue_df.loc[dengue_df[col].isnull(), col] = 9


Removing columns 'ID_AGRAVO', because it has only 1 fixed value: `A90`

In [20]:
dengue_df = dengue_df.drop(columns=['ID_AGRAVO'])

In [51]:
columns_to_be_filled = [
    col
    for col in dengue_df.columns
    if col not in exam_cols
    and 'DT_' not in str(col) # for datetime columns it doesn't make sense
    and not 'CS_SEXO'.__eq__(str(col))
]

In [42]:
dtypes = {
    'SEM_NOT': 'int32',
    'NU_ANO': 'int16',
    'SG_UF_NOT': 'category',
    'ID_MUNICIP': 'category',
    'ID_REGIONA': 'category',
    'ID_UNIDADE': 'category',
    'SEM_PRI': 'int32',
    'NU_IDADE_N': 'int8',
    'CS_SEXO': 'category',
    'CS_GESTANT': 'category',
    'CS_RACA': 'category',
    'CS_ESCOL_N': 'category',
    'SG_UF': 'category',
    'ID_MN_RESI': 'category',
    'ID_RG_RESI': 'category',
    'ID_PAIS': 'category',
    'FEBRE': 'category',
    'MIALGIA': 'category',
    'CEFALEIA': 'category',
    'EXANTEMA': 'category',
    'VOMITO': 'category',
    'NAUSEA': 'category',
    'DOR_COSTAS': 'category',
    'CONJUNTVIT': 'category',
    'ARTRITE': 'category',
    'ARTRALGIA': 'category',
    'PETEQUIA_N': 'category',
    'LEUCOPENIA': 'category',
    'LACO': 'category',
    'DOR_RETRO': 'category',
    'DIABETES': 'category',
    'HEMATOLOG': 'category',
    'HEPATOPAT': 'category',
    'RENAL': 'category',
    'HIPERTENSA': 'category',
    'ACIDO_PEPT': 'category',
    'AUTO_IMUNE': 'category',
    'RESUL_SORO': 'category',
    'RESUL_NS1': 'category',
    'RESUL_VI_N': 'category',
    'RESUL_PCR_': 'category',
    'HISTOPA_N': 'category',
    'IMUNOH_N': 'category',
    'HOSPITALIZ': 'category',
    'TPAUTOCTO': 'category',
    'COUFINF': 'category',
    'COPAISINF': 'category',
    'COMUNINF': 'category',
    'CLASSI_FIN': 'object',
    'EVOLUCAO': 'category',
}

### Setting dtypes to columns:

In [43]:
date_cols = ['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_NASC', 'DT_INVEST', 'DT_ENCERRA']
for col in date_cols:
    if col in dengue_df.columns:
        dengue_df[col] = pd.to_datetime(dengue_df[col], errors='coerce')

dengue_df['SEM_PRI'] = dengue_df['SEM_PRI'].apply(lambda x: x.replace('-', '') if isinstance(x, str) else x)
dengue_df = dengue_df.astype(dtypes)

In [44]:
dengue_df

,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,SEM_PRI,DT_NASC,...,HISTOPA_N,IMUNOH_N,HOSPITALIZ,TPAUTOCTO,COUFINF,COPAISINF,COMUNINF,CLASSI_FIN,EVOLUCAO,DT_ENCERRA
0,2018-05-07,201819,2018,25,251620,1426.0,2504537.0,2018-05-06,201819,1995-07-06,...,4.0,4.0,9.0,9.0,9.0,9.0,9,Dengue,1.0,2018-07-01
1,2018-02-02,201805,2018,25,250840,1426.0,2592266.0,2018-01-29,201805,2001-03-21,...,4.0,4.0,9.0,9.0,9.0,9.0,9,Discarded/Inconclusive,9.0,2018-04-04
2,2018-02-05,201806,2018,25,250840,1426.0,2606399.0,2018-02-03,201805,1982-07-05,...,4.0,4.0,9.0,9.0,9.0,9.0,9,Dengue,1.0,2018-03-26
3,2018-01-22,201804,2018,25,250840,1426.0,2606399.0,2018-01-05,201801,1973-01-08,...,4.0,4.0,9.0,9.0,9.0,9.0,9,Dengue,1.0,2018-02-27
4,2018-05-23,201821,2018,25,251080,1422.0,3232514.0,2018-05-21,201821,NaT,...,4.0,4.0,9.0,9.0,9.0,9.0,9,Dengue,9.0,2018-07-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12407121,2016-09-01,201635,2016,14,140060,9.0,2320789.0,2016-08-27,201634,1961-10-14,...,4.0,4.0,2.0,9.0,9.0,9.0,9,Discarded/Inconclusive,1.0,2016-09-01
12407122,2016-01-13,201602,2016,14,140017,9.0,2320258.0,2016-01-05,201601,2000-12-01,...,4.0,4.0,9.0,1.0,14.0,1.0,140017.0,Dengue,1.0,2016-03-14
12407123,2016-01-13,201602,2016,14,140017,9.0,2320258.0,2016-01-06,201601,2003-02-06,...,4.0,4.0,2.0,9.0,9.0,9.0,9,Discarded/Inconclusive,1.0,2016-03-14
12407124,2016-01-14,201602,2016,14,140017,9.0,2320258.0,2016-01-10,201602,1968-11-21,...,4.0,4.0,2.0,9.0,9.0,9.0,9,Discarded/Inconclusive,1.0,2016-03-15


In [49]:
dengue_df.dtypes

DT_NOTIFIC    datetime64[ns]
SEM_NOT                int32
NU_ANO                 int16
SG_UF_NOT           category
ID_MUNICIP          category
ID_REGIONA          category
ID_UNIDADE          category
DT_SIN_PRI    datetime64[ns]
SEM_PRI                int32
DT_NASC       datetime64[ns]
NU_IDADE_N              int8
CS_SEXO             category
CS_GESTANT          category
CS_RACA             category
CS_ESCOL_N          category
SG_UF               category
ID_MN_RESI          category
ID_RG_RESI          category
ID_PAIS             category
DT_INVEST     datetime64[ns]
FEBRE               category
MIALGIA             category
CEFALEIA            category
EXANTEMA            category
VOMITO              category
NAUSEA              category
DOR_COSTAS          category
CONJUNTVIT          category
ARTRITE             category
ARTRALGIA           category
PETEQUIA_N          category
LEUCOPENIA          category
LACO                category
DOR_RETRO           category
DIABETES      

In [50]:
dengue_df.CS_SEXO

0           F
1           F
2           M
3           M
4           M
           ..
12407121    F
12407122    F
12407123    F
12407124    F
12407125    M
Name: CS_SEXO, Length: 12407126, dtype: category
Categories (4, object): [9, 'F', 'I', 'M']